In [2]:
from kubeflow import fairing
from kubeflow.fairing.kubernetes import utils as k8s_utils

PRIVATE_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'

fairing.config.set_preprocessor(
    'python', 
     # command = ['python'],  # default: python
     input_files = ['00-python-file-to-fairing.py'],     
)

fairing.config.set_builder(
    'append',
    # base_image = f'{PRIVATE_REGISTRY}/kf-base:latest', # 사전준비에서 마련한 Base Image
    base_image = 'tensorflow/tensorflow:2.0.3-gpu-py3',
    registry = PRIVATE_REGISTRY,
    image_name='my-02-python-file-fairing', 
    push=True
)

fairing.config.set_deployer(
    'job',
    namespace='myspace',
    pod_spec_mutators=[
        k8s_utils.get_resource_mutator(cpu=1, memory=5)]
)

fairing.config.run()

[I 210103 17:16:15 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.base.BasePreProcessor object at 0x7fe87f5d5b00>
[I 210103 17:16:15 config:136] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7fe87f5d5ac8>
[I 210103 17:16:15 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7fe87f5d5ba8>
[W 210103 17:16:15 append:50] Building image using Append builder...
[I 210103 17:16:15 base:107] Creating docker context: /tmp/fairing_context_z3rb6lvv
[I 210103 17:16:15 docker_creds_:234] Loading Docker credentials for repository 'tensorflow/tensorflow:2.0.3-gpu-py3'
[W 210103 17:16:17 append:54] Image successfully built in 2.088375209015794s.
[W 210103 17:16:17 append:94] Pushing image kubeflow-registry.default.svc.cluster.local:30000/my-02-python-file-fairing:5E86763D...
[I 210103 17:16:17 docker_creds_:234] Loading Docker credentials for repository 'kubeflow-registry.default.svc.cluster.local:30000/my-02-python-file

11493376/11490434 [==============================] - 0s 0us/step
2021-01-03 17:16:46.988468: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-01-03 17:16:46.988531: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (-1)
2021-01-03 17:16:46.988568: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fairing-job-zbbg2-8bf9k): /proc/driver/nvidia/version does not exist
2021-01-03 17:16:46.988853: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-01-03 17:16:46.997452: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2021-01-03 17:16:46.998183: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x3addaa0 executing computations on platform Host. Devices:
2021-01-03 17:16:

[W 210103 17:17:22 job:173] Cleaning up job fairing-job-zbbg2...


(<kubeflow.fairing.preprocessors.base.BasePreProcessor at 0x7fe87f5d5b00>,
 <kubeflow.fairing.deployers.job.job.Job at 0x7fe87f5d5ba8>)

# Image 확인
- Host에서 실행
- fairing에서 난수로 생성한 tag를 변경해서 실행할 것 (예: E2683407)

### Image pull
```sh
docker pull kubeflow-registry.default.svc.cluster.local:30000/my-02-python-file-fairing:E2683407
```

### Image inspect
```sh
docker inspect kubeflow-registry.default.svc.cluster.local:30000/my-02-python-file-fairing:E2683407
```
Cmd 항목을 확인하고 faring 코드대로 image build 되었음을 확인
```json
...
            "Cmd": [
                "python",
                "/app/00-python-file-to-fairing.py"
            ],
...
```